# Applying the Naive Bayes' Algorithm to build a Spam Filter

In this project I am applying the Naive Bayes' Algorithm to create a spam filter. To do that, I'll use the multinomial Naive Bayes' algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the __[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)__. The data collection process is described in more details on __[this page](https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition)__ page, where you can also find some of the authors' papers. 

## Naive Bayes' Algorithm:

Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ theorem with the “naive” assumption of conditional independence between every pair of features given the value of the class variable. Bayes’ theorem states the following relationship, given class variable $y$ and dependent feature vector $x_1$ through $x_n$:

$$
\begin{aligned}
P(y|x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots, x_n | y)}{P(x_1, \dots, x_n)}
\end{aligned}
$$

Using the naive conditional independence assumption that:
$$
\begin{aligned}
P(x_i|y,x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i|y),
\end{aligned}
$$

for all i, this relationship is simplified to:

$$
\begin{aligned}
P(y|x_1, \dots, x_n) = \frac{P(y) \prod_{i=1} ^n P(x_i|y)}{P(x_1, \dots, x_n)}
\end{aligned}
$$

Since $P(x_1, \dots , x_n)$ is constant given the input, we can use the following classification rule:#

$$
\begin{aligned}
P(y|x_1, \dots, x_n) \approx P(y) \prod_{i=1} ^n P(x_i|y)
\end{aligned}
$$

$$
\begin{aligned}
\Downarrow
\end{aligned}
$$

$$
\begin{aligned}
\widetilde{y} = arg max_y P(y) \prod_{i=1} ^n P(x_i|y)
\end{aligned}
$$

and we can use Maximum A Posteriori (MAP) estimation to estimate $P(y)$ and $P(x_i|y)$; the former is then the relative frequency of class $y$ in the training set.

The different naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution of $P(x_i|y)$.

In spite of their apparently over-simplified assumptions, naive Bayes classifiers have worked quite well in many real-world situations, famously document classification and spam filtering. They require a small amount of training data to estimate the necessary parameters. (For theoretical reasons why naive Bayes works well, and on which types of data it does, see the references below.)

Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that each distribution can be independently estimated as a one dimensional distribution. This in turn helps to alleviate problems stemming from the curse of dimensionality.

Source:
H. Zhang (2004). __[The optimality of Naive Bayes](https://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf)__. Proc. FLAIRS.

In [272]:
import pandas as pd
import numpy as np

# Rawdata

In [273]:
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['Label', 'SMS'])
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [275]:
df['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Roughly 87% of the messages are ham and about 13% are spam.

## Splitting Data into Test- and Training Datasets

Once the spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A training set, which we'll use to "train" the computer how to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

In [276]:
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['Label', 'SMS']) # reload data just in case
df_randomized = df.sample(frac=1, random_state=1) # randomize dataframe

# Calculate index for split - 80% of dataset
training_test_index = round(len(df_randomized) * 0.8)

df_training = df_randomized[:training_test_index].reset_index(drop=True) # 80% of the data for training dataset
df_testing = df_randomized[training_test_index:].reset_index(drop=True) # 20% of the data for testing dataset

# print shapes
print(df_training.shape)
print(df_testing.shape)

df_training.head()

(4458, 2)
(1114, 2)


,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.



In [277]:
df_training['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [278]:
df_testing['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Data checks out!

# Cleaning the Dataset

- removing punctutation
- set to lower case
- change format of table:
    - create list of unique vocabulary
    - change table to matrix that indicates the times each vocabular occurs in the e-mail

In [279]:
df_training['SMS'] = df_training['SMS'].str.replace('\W', ' ') # remove punctuation
df_training['SMS'] = df_training['SMS'].str.lower() # set lower case
df_training.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [280]:
df_training['SMS'] = df_training['SMS'].str.split()

vocabulary = []
for sms in df_training['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))
vocabulary[:10]

['blanked',
 'jolly',
 'nokia6600',
 'doggin',
 'answers',
 'ph',
 'fromwrk',
 'categories',
 'lasagna',
 '169']

In [263]:
len(vocabulary)

7783

In [264]:
# create dictionary that counts how many times each word appears per email:

# empty dictionary of required length
word_counts_per_sms = {unique_word: [0] * len(df_training['SMS']) for unique_word in vocabulary}

for i, words in enumerate(df_training['SMS']): # iterate through list of words
    for word in words: # iterate through each word for list of words
        word_counts_per_sms[word][index] +=1

# transform dict to dataframe:
df_word_counts = pd.DataFrame(word_counts_per_sms)
df_word_counts.head()

,blanked,jolly,nokia6600,doggin,answers,ph,fromwrk,categories,lasagna,169,...,bucks,lunchtime,computers,ger,racing,poop,owe,die,stairs,panties
0,1,2,2,1,4,2,1,1,1,1,...,6,1,1,1,1,2,1,7,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [265]:
# concat training dataset to the one I just built, to have original dataframe + vocabulary matrix in one dataset
df_training_clean = pd.concat([df_training, df_word_counts], axis = 1)
df_training_clean.head()

,Label,SMS,blanked,jolly,nokia6600,doggin,answers,ph,fromwrk,categories,...,bucks,lunchtime,computers,ger,racing,poop,owe,die,stairs,panties
0,ham,"[yep, by, the, pretty, sculpture]",1,2,2,1,4,2,1,1,...,6,1,1,1,1,2,1,7,1,1
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Determining Constants of the Spam Filter

To classify the messages I'll calculate the following probabilites:

$$
\begin{aligned}
P(Spam|w_1, w_2, \dots, w_n) \approx P(Spam) \cdot \prod _1 ^n P(w_i|Spam)
\end{aligned}
$$

$$
\begin{aligned}
P(Ham|w_1, w_2, \dots, w_n) \approx P(Ham) \cdot \prod _1 ^n P(w_i|Ham)
\end{aligned}
$$

To calculate $P(wi|Spam)$ and $P(wi|Ham)$:

$$
\begin{aligned}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{aligned}
$$

$$
\begin{aligned}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{aligned}
$$

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$

- $N_{Spam}$ is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.

- $N_{Ham}$ is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.

I'll also use Laplace smoothing and set $\alpha = 1$

In [266]:
# P(Spam) / P(Ham):
# filter by spam / ham
df_spam = df_training_clean[df_training_clean['Label'] == 'spam']
df_ham = df_training_clean[df_training_clean['Label'] == 'ham']

# p(spam) = len(spam dataframe) / len(total dataframe)
p_spam = len(df_spam) / len(df_training_clean)
p_ham = len(df_ham) / len(df_training_clean)

print('P(Spam) = ' + str(p_spam))
print('P(Ham) = ' + str(p_ham))

# N_spam:
n_words_per_spam_message = df_spam['SMS'].apply(len) # apply len function to each list of strings
n_spam = n_words_per_spam_message.sum() # sum each len to get total no of words

# N_ham
n_words_per_ham_message = df_ham['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

print('N_spam = ' + str(n_spam))
print('P_ham = ' + str(n_ham))

# N_vocabulary = list of unique words
n_vocabulary = len(vocabulary)
print('N_vocabulary = ' + str(n_vocabulary))

# Laplace smoothing
alpha = 1

P(Spam) = 0.13458950201884254
P(Ham) = 0.8654104979811574
N_spam = 15190
P_ham = 57237
N_vocabulary = 7783


## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
\begin{aligned}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{aligned}
$$

$$
\begin{aligned}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{aligned}
$$


reminder:  $N_{wi|Ham}$ is equal to the number of times the word wi occurs in all the ham messages.

In [267]:
# initiating empty dictionaries
parameters_spam = {unique_word : 0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# calculating parameters p(w_i|spam) and p(w_i|ham):
for word in vocabulary: # for each word in vocabulary list
    
    # spam
    n_wi_spam = df_spam[word].sum() # find how often it occurs in df_spam (remember matrix format allows for df.sum())
    p_wi_spam = (n_wi_spam + alpha) / (n_spam + (alpha * n_vocabulary)) # use formula above to calculate p(wi|ham)
    parameters_spam[word] = p_wi_spam # append result to parameters_spam
    
    # ham
    n_wi_ham = df_ham[word].sum()
    p_wi_ham = (n_wi_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    parameters_ham[word] = p_wi_ham


# Creating the Spam Filter

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.



In [268]:
import re

def classify(message):
    
    '''
    input: message = list of strings, cleaned as specified in the beginning of this notebook
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    # initial values:
    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham 
    
    for word in message: # iterate through message
        if word in parameters_spam: # if word in parameters_spam dict
            p_spam_given_message *= parameters_spam[word] # multiply p_spam_given_message with value in dict
            
        if word in parameters_ham: # if word in parameters_ham dict
            p_ham_given_message *= parameters_ham[word] # multiply p_ham_given_message with value in dict

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
        
    return


# two test messages:
# obviously spam:
test_spam = 'WINNER!! This is the secret code to unlock the money: C3421.'
test_ham = 'Sounds good, Tom, then see u there'

print('test message 1: ' + test_spam)
classify(test_spam)

print('-' * 45)
print('test message 2: ' + test_ham)
classify(test_ham)

test message 1: WINNER!! This is the secret code to unlock the money: C3421.
P(Spam|message): 7.55182248895403e-41
P(Ham|message): 3.2853927642755508e-24
Label: Ham
---------------------------------------------
test message 2: Sounds good, Tom, then see u there
P(Spam|message): 3.9855402384039956e-32
P(Ham|message): 7.53202812851122e-21
Label: Ham


## Somethin is OFF!!

The first message should be classified as spam

# Testing the algorithm on the entire dataset:

- First, change function to return result rather than print the message
- Calculate accuracy by combining the results of the algorithm to the human results

In [269]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    
# apply function to df_test dataset
df_testing['predicted'] = df_testing['SMS'].apply(classify_test_set)
df_testing.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,ham
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


measuring the accuracy

In [270]:
correct = 0

for i, row in df_testing.iterrows():
    if row['Label'] == row['predicted']:
        correct +=1

total = len(df_testing['SMS']) # total no of messages in test dataset
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 966
Incorrect: 148
Accuracy: 0.8671454219030521


# Conclusion:

The solution posted on GitHub sohows an accoracy of 96%